In [38]:
# Strongly-local code for minimizing the HCL objective
# Implemented with the thresholded linear hyperedge splitting penalty.

include("../include/HyperLocal/Helper_Functions.jl")
include("../include/HyperLocal/maxflow.jl")
include("../src/SparseCard.jl")

function CoreCut(G::SparseMatrixCSC{Float64,Int64}, tau::Float64, epsilon::Float64,
    to_exclude::Vector{Int64})
    
    N = size(G)[1]
    d = sum(G, dims=2)
    volG = sum(d)
    R = setdiff(collect(1:N), to_exclude)

    Edges = [collect(1:N)]
    EdgesW = [(0.5 * collect(0:N) .* reverse(collect(0:N))/N)[1:Int64((floor(N/2)+1))]]


    A = SymmetricCard_reduction(Edges, EdgesW, N, epsilon, false)
    A[1:N,1:N] =  A[1:N,1:N] + G
    A_N = size(A)[1]


    sVec = zeros(A_N)
    sVec[1:N] = tau * ones(N) + d
    
    tVec = zeros(A_N)
    tVec[to_exclude] .= Inf

    BestS = R
    alpha_old = Inf
    alpha_new = CoreCutScore(G, R, tau)
    alphaBest = alpha_new

    while alphaBest < alpha_old
        print("Old CoreCutScore: ")
        println(alpha_old)
        alpha_old = alphaBest
        
        sVec = alphaBest.*sVec
       # println(typeof(sVec))
        println(sVec)
        println("maxflow started")
        F = maxflow(A,vec(sVec),tVec,0)
        Src = source_nodes_min(F)[2:end].-1
        S = intersect(1:N,Src)
        alpha_new = CoreCutScore(G, S, tau)
        print("New CoreCutScore: ")
        println(alpha_new) 

        if alpha_new < alphaBest
            alphaBest = alpha_new
            BestS = S
            print("S updated: ")
            println(length(S))
        end
    end

    return BestS, alphaBest
end

function CoreCutScore(G::SparseMatrixCSC{Float64,Int64},S::Vector{Int64}, tau::Float64)
    cut, vol, edges, cond = set_stats(G, S, sum(G.nzval))
    N = size(G)[1]
    s_len = length(S)

    return  (cut+tau*s_len*(N-s_len)/N)/(vol+tau*s_len)
end

CoreCutScore (generic function with 2 methods)

In [3]:
using MatrixMarket
using StatsBase 
M = MatrixMarket.mmread("minnesota.mtx")
M_coord = MatrixMarket.mmread("minnesota_coord.mtx")

2642×2 Matrix{Float64}:
 -97.207  49.001
 -96.801  49.0
 -95.957  49.0
 -95.931  49.0
 -95.766  49.0
 -95.378  48.999
 -97.2    48.972
 -97.236  48.965
 -95.331  48.914
 -95.768  48.849
 -95.768  48.848
 -95.92   48.833
 -95.09   48.799
   ⋮      
 -91.871  43.501
 -94.994  43.501
 -92.369  43.5
 -96.209  43.5
 -96.198  43.5
 -94.088  43.5
 -95.694  43.5
 -95.643  43.5
 -92.944  43.5
 -93.247  43.5
 -93.353  43.5
 -93.493  43.499

In [4]:
to_exclude = sample(collect(1:2642), 10,replace = false)

10-element Vector{Int64}:
 1811
 1756
 1635
 1983
  124
  329
 1723
  197
  944
  617

In [39]:
CoreCut(SparseMatrixCSC{Float64, Int64}(M),0.5, 0.0,to_exclude)

Old CoreCutScore: Inf
[0.006076201699156079, 0.006076201699156079, 0.010127002831926798, 0.006076201699156079, 0.006076201699156079, 0.006076201699156079, 0.014177803964697519, 0.006076201699156079, 0.014177803964697519, 0.014177803964697519, 0.014177803964697519, 0.014177803964697519, 0.010127002831926798, 0.006076201699156079, 0.014177803964697519, 0.014177803964697519, 0.014177803964697519, 0.006076201699156079, 0.014177803964697519, 0.014177803964697519, 0.014177803964697519, 0.010127002831926798, 0.010127002831926798, 0.010127002831926798, 0.006076201699156079, 0.014177803964697519, 0.014177803964697519, 0.010127002831926798, 0.014177803964697519, 0.014177803964697519, 0.010127002831926798, 0.018228605097468237, 0.014177803964697519, 0.006076201699156079, 0.018228605097468237, 0.010127002831926798, 0.010127002831926798, 0.014177803964697519, 0.018228605097468237, 0.010127002831926798, 0.010127002831926798, 0.010127002831926798, 0.010127002831926798, 0.010127002831926798, 0.0141778

LoadError: ArgumentError: reducing over an empty collection is not allowed